Description (2025-01-03):<br>
Authors:<br>
Rui Gao and Alfonso Torres-Rua<br>
Utah State University

This script contains components:
- Thermal and spectral image processing.
- Extract canopy temperature at the 0.6 m pixel resolution.
- Attach the minimum air temperature, the air temperature at the UAV flight time, and the difference between these two values for each canopy temperature.
- Import the trained XGBoost model (called "xgb_tt.pkl").
- Estimate the leaf water potential for each 0.6 m pixel (canopy pixel).
- Providing a leaf water potential map at the end (in TIFF format).

Contacts:<br>
Please contact Rui if you have questions about this model.
- rui.ray.gao@gmail.com
- rui.gao@usu.edu

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import pickle
import rasterio
from rasterio.enums import Resampling
from rasterio.mask import mask

# Set up paths and import custom functions
import functions_4_lwp as lwp

# Load XGBoost model
xgb_model = r"xgb_tt.pkl"
with open(xgb_model, 'rb') as file:
    model = pickle.load(file)

In [ ]:
# File paths
current_dir = os.getcwd()
file_rgb = os.path.join(current_dir,"input_data","Demo_Input_VNIR.tif")
file_tr = os.path.join(current_dir,"input_data","Demo_Input_TIR.tif")
dir_output = os.path.join(current_dir,"onput_data","Demo_Output")
dir_table_output = os.path.join(current_dir,"input_data","output_data")

# Create output folder
lwp.FolderCreater(dir_output)

# File parameters and constants
file_number = 'Demo_Output'
resolution_high = 0.15
resolution_low = 0.60
threshold_ndvi = 0.65
NoDataValue = -9999
# From the EC-flux tower or local air temperature sensor
# # AT: air temperature at the flight time;
# AT_min: the minimum air temperature from the historical 24 hours;
# AT_diff: the air temperature difference between above two temperatures.
AT, AT_min, AT_diff = 29.99, 16.25, 13.74  

In [12]:
# Step 1: Resample the Thermal and RGBNIR Images
# Resample Thermal to low resolution
resample_tr = f"{file_number}_res_tr.tif"
lwp.resample_raster(
    input_file=file_tr,
    output_file=f"{dir_output}\\{resample_tr}",
    new_resolution=resolution_low,
    resample_method='cubic'
)

# Resample RGBNIR to low resolution
resample_rgb = f"{file_number}_res_rgb.tif"
lwp.resample_raster(
    input_file=file_rgb,
    output_file=f"{dir_output}\\{resample_rgb}",
    new_resolution=resolution_low,
    resample_method='cubic'
)

# Step 2: Calculate NDVI from Resampled RGBNIR
ndvi_output_file = f"{file_number}_ndvi.tif"
with rasterio.open(f"{dir_output}\\{resample_rgb}") as src:
    red_band = src.read(1).astype('float32')
    nir_band = src.read(4).astype('float32')
    ndvi = np.where((nir_band + red_band) == 0, 0, (nir_band - red_band) / (nir_band + red_band))

    # Save NDVI as a new file
    ndvi_meta = src.meta.copy()
    ndvi_meta.update({
        "count": 1,
        "dtype": "float32"
    })
    with rasterio.open(f"{dir_output}\\{ndvi_output_file}", 'w', **ndvi_meta) as dst:
        dst.write(ndvi, 1)
print("NDVI calculated and saved to:", ndvi_output_file)

# Step 3: Clip NDVI to Thermal Image Extent
output_clipped_ndvi = f"{file_number}_clp_ndvi.tif"
with rasterio.open(f"{dir_output}\\{resample_tr}") as thermal_src:
    thermal_bounds = thermal_src.bounds
    thermal_transform = thermal_src.transform
    thermal_crs = thermal_src.crs
    thermal_height, thermal_width = thermal_src.shape

# Use the thermal bounds to clip the NDVI image
with rasterio.open(f"{dir_output}\\{ndvi_output_file}") as ndvi_src:
    bbox = [{
        'type': 'Polygon',
        'coordinates': [[
            (thermal_bounds.left, thermal_bounds.bottom),
            (thermal_bounds.left, thermal_bounds.top),
            (thermal_bounds.right, thermal_bounds.top),
            (thermal_bounds.right, thermal_bounds.bottom),
            (thermal_bounds.left, thermal_bounds.bottom)
        ]]
    }]
    out_image, out_transform = mask(ndvi_src, bbox, crop=True)
    out_meta = ndvi_src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": thermal_height,
        "width": thermal_width,
        "transform": out_transform,
        "crs": thermal_crs
    })
    with rasterio.open(f"{dir_output}\\{output_clipped_ndvi}", 'w', **out_meta) as dst:
        dst.write(out_image)
print("NDVI image clipped to match the thermal image extent and saved to:", output_clipped_ndvi)

# Step 4: Prepare Data for Model Prediction
with rasterio.open(f"{dir_output}\\{resample_tr}") as src_tr, rasterio.open(f"{dir_output}\\{output_clipped_ndvi}") as src_ndvi:
    if src_tr.shape != src_ndvi.shape:
        raise ValueError("Dimension mismatch between thermal and NDVI images.")
    Tr_flat = src_tr.read(1).flatten()
    NDVI_flat = src_ndvi.read(1).flatten()
    df = pd.DataFrame({
        'Tr': Tr_flat,
        'NDVI': NDVI_flat,
        'AT': AT,
        'AT_min': AT_min,
        'AT_diff': AT_diff
    })
print("DataFrame with Tr and NDVI values was prepared!")

 # Step 5: Make Estimations
# Predict LWP based on Tr, AT, AT_min, and AT_diff
X = df[['Tr', 'AT', 'AT_min', 'AT_diff']]
df['LWP'] = model.predict(X)
print("Predictions added to DataFrame:")

# Step 6: Filter LWP Values Based on Conditions
df.loc[(df['Tr'] < 0) | (df['Tr'] > 60), 'LWP'] = -999
df.loc[(df['NDVI'] < threshold_ndvi) | (df['NDVI'] > 1), 'LWP'] = -999
print("DataFrame was filtered!")

# Step 7: Save LWP Predictions to Raster
LWP_data = df['LWP'].values.reshape(src_tr.shape)
ndvi_meta.update({
    "dtype": 'float32',
    "count": 1
})

img_lwp = f"{file_number}.tif"
with rasterio.open(f"{dir_output}\\{img_lwp}", 'w', **out_meta) as dst:
    dst.write(LWP_data, 1)
print("LWP image saved to:", img_lwp)

Resampling complete. Output saved to: e:\3_Research_PhD\20240514_Project_LWP\1_Manuscript\5_Zenodo_Model_Package\Model_Package\onput_data\Demo_Output\Demo_Output_res_tr.tif
Resampling complete. Output saved to: e:\3_Research_PhD\20240514_Project_LWP\1_Manuscript\5_Zenodo_Model_Package\Model_Package\onput_data\Demo_Output\Demo_Output_res_rgb.tif
NDVI calculated and saved to: Demo_Output_ndvi.tif
NDVI image clipped to match the thermal image extent and saved to: Demo_Output_clp_ndvi.tif
DataFrame with Tr and NDVI values was prepared!
Predictions added to DataFrame:
DataFrame was filtered!
LWP image saved to: Demo_Output.tif
